<a href="https://colab.research.google.com/github/Balbuenar/Deep-Learning-and-Reinforcement-Learning/blob/main/05g_DEMO_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 12ms/step - loss: 0.6825 - accuracy: 0.5556 - val_loss: 0.6253 - val_accuracy: 0.6744
Epoch 2/10
782/782 [==============================] - 9s 11ms/step - loss: 0.5943 - accuracy: 0.7123 - val_loss: 0.5588 - val_accuracy: 0.7228
Epoch 3/10
782/782 [==============================] - 9s 11ms/step - loss: 0.5193 - accuracy: 0.7544 - val_loss: 0.5162 - val_accuracy: 0.7448
Epoch 4/10
782/782 [==============================] - 9s 11ms/step - loss: 0.4680 - accuracy: 0.7835 - val_loss: 0.4897 - val_accuracy: 0.7628
Epoch 5/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4363 - accuracy: 0.8034 - val_loss: 0.4745 - val_accuracy: 0.7718
Epoch 6/10
782/782 [==============================] - 9s 11ms/step - loss: 0.4038 - accuracy: 0.8185 - val_loss: 0.4637 - val_accuracy: 0.7773
Epoch 7/10
782/782 [==============================] - 9s 11ms/step - loss: 0.3917 - accuracy: 0.8232 - val_loss: 0.4527 - val_accuracy: 0.783

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 2ms/step - loss: 0.4489 - accuracy: 0.7897
Test score: 0.4489096403121948
Test accuracy: 0.789680004119873


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [11]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [12]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 18s 21ms/step - loss: 0.6658 - accuracy: 0.5904 - val_loss: 0.5555 - val_accuracy: 0.7052
Epoch 2/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4859 - accuracy: 0.7692 - val_loss: 0.4746 - val_accuracy: 0.7712
Epoch 3/10
782/782 [==============================] - 16s 21ms/step - loss: 0.4193 - accuracy: 0.8098 - val_loss: 0.4424 - val_accuracy: 0.7929
Epoch 4/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3791 - accuracy: 0.8293 - val_loss: 0.4250 - val_accuracy: 0.8036
Epoch 5/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3421 - accuracy: 0.8557 - val_loss: 0.4136 - val_accuracy: 0.8069
Epoch 6/10
782/782 [==============================] - 17s 21ms/step - loss: 0.3211 - accuracy: 0.8622 - val_loss: 0.3924 - val_accuracy: 0.8254
Epoch 7/10
782/782 [==============================] - 16s 21ms/step - loss: 0.2984 - accuracy: 0.8774 - val_loss: 0.3831 - val_accuracy:

In [15]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 14s 17ms/step - loss: 0.6681 - accuracy: 0.5667 - val_loss: 0.6066 - val_accuracy: 0.6854
Epoch 2/10
782/782 [==============================] - 13s 16ms/step - loss: 0.5038 - accuracy: 0.7585 - val_loss: 0.4644 - val_accuracy: 0.7814
Epoch 3/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4254 - accuracy: 0.8063 - val_loss: 0.4347 - val_accuracy: 0.7906
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3870 - accuracy: 0.8299 - val_loss: 0.4070 - val_accuracy: 0.8150
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3568 - accuracy: 0.8465 - val_loss: 0.4032 - val_accuracy: 0.8125
Epoch 6/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3408 - accuracy: 0.8536 - val_loss: 0.4004 - val_accuracy: 0.8168
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3246 - accuracy: 0.8627 - val_loss: 0.3760 - val_accuracy:

In [19]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2966 - accuracy: 0.8752 - val_loss: 0.3582 - val_accuracy: 0.8410
Epoch 2/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2913 - accuracy: 0.8787 - val_loss: 0.3665 - val_accuracy: 0.8387
Epoch 3/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2868 - accuracy: 0.8805 - val_loss: 0.3579 - val_accuracy: 0.8418
Epoch 4/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2832 - accuracy: 0.8810 - val_loss: 0.3612 - val_accuracy: 0.8428
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2804 - accuracy: 0.8835 - val_loss: 0.3644 - val_accuracy: 0.8399
Epoch 6/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2776 - accuracy: 0.8826 - val_loss: 0.3613 - val_accuracy: 0.8424
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2751 - accuracy: 0.8850 - val_loss: 0.3589 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation